**Nettoyage des données et création des CSV**


In [1]:
import os, re, unicodedata
import hashlib
import pandas as pd
import numpy as np


In [2]:
# Import des données
df_f = pd.read_csv("/content/Sneakers_femmes_decath.csv",  sep=None, engine="python", encoding="utf-8-sig", on_bad_lines="skip")
df_m = pd.read_csv("/content/Sneakers_hommes_decath.csv",  sep=None, engine="python", encoding="utf-8-sig", on_bad_lines="skip")
df_k = pd.read_csv("/content/Sneakers_enfants_decath.csv", sep=None, engine="python", encoding="utf-8-sig", on_bad_lines="skip")


In [3]:
# vérification des données
print("Femmes:", df_f.shape)
print("Hommes:", df_m.shape)
print("Enfants:", df_k.shape)

Femmes: (1185, 10)
Hommes: (1239, 10)
Enfants: (419, 10)


In [4]:
# Mapping des colonnes aux noms de la base
COLMAP = {
    "Prix_actuel":"price_cur", "Prix_initial":"price_init",
    "Baisse_du_prix":"discount", "Durée_de_vente":"sale_duration",
    "Marque":"brand", "Description":"name",
    "Commentaire":"comment", "Livraison":"shipping",
    "Image_URL":"image_url", "URL_de_la_page_détaillée":"url"
}

def rename_cols(df, gender_code):
    df = df.rename(columns=COLMAP)
    df["gender_raw"] = gender_code
    # on force toutes les colonnes standard
    for v in COLMAP.values():
        if v not in df.columns:
            df[v] = None
    return df[list(COLMAP.values()) + ["gender_raw"]]

In [5]:
df_f_std = rename_cols(df_f, "F")
df_m_std = rename_cols(df_m, "M")
df_k_std = rename_cols(df_k, "KID")

In [6]:
df = pd.concat([df_f_std, df_m_std, df_k_std], ignore_index=True)
print(df.shape)
df.head(3)

(2843, 11)


,price_cur,price_init,discount,sale_duration,brand,name,comment,shipping,image_url,url,gender_raw
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F
1,"49,99€",NaN,NaN,NaN,NEWFEEL,Chaussures imperméables de marche nordique NW ...,(874),Livraison en 48h,https://contents.mediadecathlon.com/p1867762/k...,https://www.decathlon.fr/p/chaussures-impermea...,F
2,"89,99€",NaN,NaN,NaN,KIPSTA,Chaussures de basketball homme/femme - CANAVER...,(12),NaN,https://contents.mediadecathlon.com/p2969847/k...,https://www.decathlon.fr/p/chaussure-de-basket...,F


In [7]:
# Déclaration des fonctions utilitaires
def slugify_local(s: str) -> str:
    """
    Transforme un texte en identifiant "slug" utilisable dans une URL ou une DB.
    - supprime les accents
    - met en minuscule
    - remplace tout ce qui n'est pas [a-z0-9] par '-'
    """
    s = str(s) if s is not None else ""
    s = unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode().lower()
    s = re.sub(r"[^a-z0-9]+", "-", s).strip("-")
    return s

In [8]:
def norm_txt(s: str) -> str:
    """
    Nettoie un texte :
    - supprime accents
    - minuscule
    - supprime espaces inutiles
    """
    if not isinstance(s, str): return ""
    s = unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode().lower().strip()
    return s

In [9]:
def price_to_eur(v):
    """
    Convertit un prix texte '49,99€' -> float 49.99
    - enlève tout sauf chiffres, virgule et point
    - remplace virgule par point
    - convertit en float arrondi à 2 décimales
    """
    if pd.isna(v):
        return None
    s = str(v)
    digits = re.sub(r"[^0-9,\.]", "", s)  # garde uniquement chiffres, . et ,
    if not digits:
        return None
    try:
        return round(float(digits.replace(",", ".")), 2)
    except:
        return None

In [10]:
#  Déduplication

def fingerprint_row(r):
    """
    Crée une empreinte unique d’un produit :
    - slug du nom
    - marque normalisée
    - genre
    Sert si l’URL n’est pas dispo pour dédoublonner.
    """
    nm = slugify_local(str(r.get("name","")))
    br = norm_txt(str(r.get("brand","")))
    gd = str(r.get("gender_raw",""))
    return f"{nm}|{br}|{gd}"


In [11]:
# Appliquer le nettoyage

# 1) supprimer doublons : URL si dispo sinon fingerprint
df["fp"] = df.apply(lambda r: r["url"] if pd.notna(r["url"]) and str(r["url"]).strip()
                    else fingerprint_row(r), axis=1)
df = df.sort_values("fp").drop_duplicates("fp", keep="first").drop(columns=["fp"])

In [12]:
# 2) normaliser marque, prix, slug
df["brand_norm"]  = df["brand"].fillna("").apply(lambda b: norm_txt(b).title())
df["price_eur"]   = df["price_cur"].apply(price_to_eur)
df["price_cents"] = (df["price_eur"] * 100).round().astype("Int64")
df["product_slug"] = df["name"].fillna("").apply(slugify_local)

In [13]:
# 3) ajouter genre lisible et pour DB
GENDER_MAP_DB = {"F":"woman","M":"man","KID":"child"}
GENDER_MAP_LABEL = {"F":"femme","M":"homme","KID":"enfant"}
df["genre_age"] = df["gender_raw"].map(GENDER_MAP_LABEL)
df["gender_db"] = df["gender_raw"].map(GENDER_MAP_DB)

In [14]:
print("Après nettoyage :", df.shape)
df.head(5)

Après nettoyage : (2231, 17)


,price_cur,price_init,discount,sale_duration,brand,name,comment,shipping,image_url,url,gender_raw,brand_norm,price_eur,price_cents,product_slug,genre_age,gender_db
1286,"84,99€",NaN,NaN,NaN,ADIDAS,ADIDAS TERREX ANYLANDER IMPERMEABLE,(4),Livraison en 48h,https://contents.mediadecathlon.com/p2784230/k...,https://www.decathlon.fr/p/adidas-terrex-anyla...,M,Adidas,84.99,8499,adidas-terrex-anylander-impermeable,homme,man
194,"84,99€",NaN,NaN,NaN,ADIDAS,ADIDAS TERREX ANYLANDER IMPERMEABLE,(4),Livraison en 48h,https://contents.mediadecathlon.com/p2769590/k...,https://www.decathlon.fr/p/adidas-terrex-anyla...,F,Adidas,84.99,8499,adidas-terrex-anylander-impermeable,femme,woman
1330,"18,99€",NaN,NaN,NaN,DECATHLON,"Baksets homme, KLNJ BE D grise",(934),Livraison en 48h,https://contents.mediadecathlon.com/p2633211/k...,https://www.decathlon.fr/p/baksets-homme-klnj-...,M,Decathlon,18.99,1899,baksets-homme-klnj-be-d-grise,homme,man
9,"26,99€",NaN,NaN,NaN,DECATHLON,BALLERINES DE MARCHE URBAINE FEMME PW 160 BR'E...,(2671),Livraison en 48h,https://contents.mediadecathlon.com/p2394347/k...,https://www.decathlon.fr/p/ballerines-marche-s...,F,Decathlon,26.99,2699,ballerines-de-marche-urbaine-femme-pw-160-br-e...,femme,woman
129,"24,99€","29,99€",-5€,*À partir du 19/07/2025 et avant le 17/09/2025,TBS,Basket de marche en toile femmeTBS Lescala beige,(10),Livraison en 48h,https://contents.mediadecathlon.com/p2872779/k...,https://www.decathlon.fr/p/basket-de-marche-en...,F,Tbs,24.99,2499,basket-de-marche-en-toile-femmetbs-lescala-beige,femme,woman


In [15]:
# Extraction et mapping des couleurs officielles

# liste des termes "couleur" qu'on accepte (termes officiels/marketing inclus)
COLOR_TERMS = [
    # noir / blanc
    "noir","noire","noirs","noires","black","\"black",
    "blanc","blanche","blancs","blanc.","blanco","white","white...",
    # bleu
    "bleu","bleue","bleues","blue","\"blue'","blueberry","marine","navy","sky","turquoises","ultramarine",
    # rouge
    "rouge","rouges","rouge...","red","bred","shattered",
    # rose
    "rose","pink","pinksicle","pinkscape","orchid",
    # vert
    "vert","verte","vert.","green","forest","olive","kaki","khaki","military","chlorophyll",
    # jaune
    "jaune","jaune...","yellow","mustard","goldenrod",
    # gris
    "gris","grise","grises","\"gray\"","grey","gray","grisport",
    # beige / neutres
    "beige","sand","camel","sanddrift","cream","ivory","angora",
    # marron
    "marron","brown","cordura","hickory",
    # orange
    "orange","corail","coral",
    # violet / pourpre
    "violet","violette","purple","bordeaux","aurora",
    # métalliques / neutres
    "argent","argenté","doré","dore","d´or","gold","golden","silver",
    # multicolore
    "multicolore","multicolores","multicolor","multi",
    # Adjectifs(peuvent être gardés seuls)
    "clair","foncé","foncée","pastel"
]

In [16]:
# Adjectifs à traiter à part
MODIFIERS = {"clair","foncé","foncée","pastel"}

In [17]:
FAMILY_MAP = {
    "noir":"noir","noire":"noir","noirs":"noir","noires":"noir","black":"noir","\"black":"noir",
    "blanc":"blanc","blanche":"blanc","blancs":"blanc","blanc.":"blanc","blanco":"blanc","white":"blanc","white...":"blanc",
    "bleu":"bleu","bleue":"bleu","bleues":"bleu","blue":"bleu","\"blue'":"bleu","blueberry":"bleu",
    "marine":"bleu","navy":"bleu","sky":"bleu","turquoises":"bleu","ultramarine":"bleu",
    "rouge":"rouge","rouges":"rouge","rouge...":"rouge","red":"rouge","bred":"rouge","shattered":"orange",
    "rose":"rose","pink":"rose","pinksicle":"rose","pinkscape":"rose","orchid":"violet",
    "vert":"vert","verte":"vert","vert.":"vert","green":"vert","forest":"vert","olive":"vert","kaki":"vert","khaki":"vert","military":"vert","chlorophyll":"vert",
    "jaune":"jaune","jaune...":"jaune","yellow":"jaune","mustard":"jaune","goldenrod":"jaune",
    "gris":"gris","grise":"gris","grises":"gris","\"gray\"":"gris","grey":"gris","gray":"gris","grisport":"gris",
    "beige":"beige","sand":"beige","camel":"beige","sanddrift":"beige","cream":"beige","ivory":"beige","angora":"beige",
    "marron":"marron","brown":"marron","cordura":"marron","hickory":"marron",
    "orange":"orange","corail":"orange","coral":"orange",
    "violet":"violet","violette":"violet","purple":"violet","bordeaux":"violet","aurora":"violet",
    "argent":"metallique","argenté":"metallique","doré":"metallique","dore":"metallique","d´or":"metallique",
    "gold":"metallique","golden":"metallique","silver":"metallique",
    "multicolore":"multicolore","multicolores":"multicolore","multicolor":"multicolore","multi":"multicolore",

# Adjectifs modificateurs:
# Quand seuls, nuance = eux-mêmes
"clair":"clair",
"foncé":"foncé",
"foncée":"foncé",
"pastel":"pastel"
}

In [18]:
# Regex insensitive à la casse, en préservant l'ordre de détection
COLOR_PATTERN = re.compile(
    r"(" + "|".join(sorted(map(re.escape, COLOR_TERMS), key=len, reverse=True)) + r")",
    re.IGNORECASE
)

def extract_colors_from_text(text: str):
    """
    Règle demandée :
    - si on trouve des couleurs "de base" ET des modificateurs → on NE garde que les couleurs de base
      ex. 'jaune clair' → ['jaune']
    - si on ne trouve QUE des modificateurs → on les garde tels quels
      ex. 'coloris clair' → ['clair']
    - si on ne trouve rien → ['NA']
    """
    if not isinstance(text, str) or not text:
        return ["NA"]
    found = [f.lower() for f in COLOR_PATTERN.findall(text)]
    found = list(dict.fromkeys(found))  # unique, conserve l'ordre

    base = [c for c in found if c not in MODIFIERS and FAMILY_MAP.get(c, "x") is not None]
    mods = [c for c in found if c in MODIFIERS]

    if base:
        return base  # on ignore les modifs quand une vraie couleur est présente
    if mods:
        return mods  # on garde les modifs si ce sont les seules infos
    return ["NA"]

In [19]:
def to_nuance(c: str) -> str:
    fam = FAMILY_MAP.get(c)
    return fam if fam is not None else "NA"

In [20]:
df["colors_list"] = df.apply(lambda r: extract_colors_from_text(f"{r['name']} {r['comment']}".strip()), axis=1)
df_colors = df.explode("colors_list").rename(columns={"colors_list":"couleur"}).reset_index(drop=True)
df_colors["nuance"] = df_colors["couleur"].apply(to_nuance)

In [21]:
# Vérification de  détection couleurs


# 0) Sécurité : vérifie que les fonctions/objets existent (sinon lève une erreur lisible)
required = ["extract_colors_from_text", "FAMILY_MAP", "COLOR_PATTERN"]
missing = [x for x in required if x not in globals()]
if missing:
    raise RuntimeError(f"Il manque ces définitions dans ton notebook : {missing}. "
                       "Re-exécute la cellule où tu as défini COLOR_TERMS/FAMILY_MAP/COLOR_PATTERN/extract_colors_from_text.")

# 1) Créer la colonne 'colors_list' si absente (détection sur name + comment)
if "colors_list" not in df.columns:
    df["colors_list"] = df.apply(
        lambda r: extract_colors_from_text(f"{r.get('name','')} {r.get('comment','')}".strip()),
        axis=1
    )

# 2) Exploser en 1 ligne par couleur et ajouter la 'nuance'
def to_nuance(c: str) -> str:
    fam = FAMILY_MAP.get((c or "").lower())
    return fam if fam is not None else "NA"

df_colors = df.explode("colors_list").rename(columns={"colors_list":"couleur"}).reset_index(drop=True)
df_colors["nuance"] = df_colors["couleur"].apply(to_nuance)

# 3) Checks

print("Aperçu des 10 premières lignes avec couleurs/nuances :")
display(df_colors[["brand","name","genre_age","couleur","nuance"]].head(10))

print("\nTop 25 des couleurs détectées :")
display(pd.Series(df_colors["couleur"]).value_counts().head(25))

print("\nTop 25 des nuances (familles) :")
display(pd.Series(df_colors["nuance"]).value_counts().head(25))

print("\nExemples où une couleur a bien été trouvée (exclut 'NA') :")
sample_found = df_colors[df_colors["couleur"].str.lower() != "NA"]\
                  [["brand","name","genre_age","couleur","nuance"]].head(15)
display(sample_found)

print("\nExemples où rien n'a été détecté (couleur = 'NA') :")
sample_unknown = df_colors[df_colors["couleur"].str.lower() == "NA"]\
                    [["brand","name","genre_age","couleur","nuance"]].head(10)
display(sample_unknown)



Aperçu des 10 premières lignes avec couleurs/nuances :


,brand,name,genre_age,couleur,nuance
0,ADIDAS,ADIDAS TERREX ANYLANDER IMPERMEABLE,homme,NA,NA
1,ADIDAS,ADIDAS TERREX ANYLANDER IMPERMEABLE,femme,NA,NA
2,DECATHLON,"Baksets homme, KLNJ BE D grise",homme,grise,gris
3,DECATHLON,BALLERINES DE MARCHE URBAINE FEMME PW 160 BR'E...,femme,noir,noir
4,TBS,Basket de marche en toile femmeTBS Lescala beige,femme,beige,beige
5,TBS,Basket de marche en toile femmeTBS Lescala marine,femme,marine,bleu
6,SKECHERS,"Basket de marche Femme, Flex Appeal 5.0 Fresh ...",femme,NA,NA
7,SKECHERS,"Basket de marche femme, Go walk Joy Violet Noir",femme,violet,violet
8,SKECHERS,"Basket de marche femme, Go walk Joy Violet Noir",femme,noir,noir
9,SKECHERS,Basket de marche Homme Edgeride Bleu,homme,bleu,bleu



Top 25 des couleurs détectées :


,count
couleur,
NA,827
noir,222
blanc,215
white,139
bleu,137
black,116
beige,86
rose,70
blue,64



Top 25 des nuances (familles) :


,count
nuance,
NA,827
blanc,381
noir,353
bleu,288
vert,129
gris,126
rose,104
beige,101
rouge,84



Exemples où une couleur a bien été trouvée (exclut 'NA') :


,brand,name,genre_age,couleur,nuance
0,ADIDAS,ADIDAS TERREX ANYLANDER IMPERMEABLE,homme,NA,NA
1,ADIDAS,ADIDAS TERREX ANYLANDER IMPERMEABLE,femme,NA,NA
2,DECATHLON,"Baksets homme, KLNJ BE D grise",homme,grise,gris
3,DECATHLON,BALLERINES DE MARCHE URBAINE FEMME PW 160 BR'E...,femme,noir,noir
4,TBS,Basket de marche en toile femmeTBS Lescala beige,femme,beige,beige
5,TBS,Basket de marche en toile femmeTBS Lescala marine,femme,marine,bleu
6,SKECHERS,"Basket de marche Femme, Flex Appeal 5.0 Fresh ...",femme,NA,NA
7,SKECHERS,"Basket de marche femme, Go walk Joy Violet Noir",femme,violet,violet
8,SKECHERS,"Basket de marche femme, Go walk Joy Violet Noir",femme,noir,noir
9,SKECHERS,Basket de marche Homme Edgeride Bleu,homme,bleu,bleu



Exemples où rien n'a été détecté (couleur = 'NA') :


,brand,name,genre_age,couleur,nuance


In [22]:
# 4) Tests unitaires simples (phrases synthétiques)
tests = [
    "Chaussures bleu foncé",
    "Baskets vert clair",
    "Tennis rouge pastel",
    "Chaussures corail",
    "Baskets military",
    "Sneakers navy",
    "Juste clair",
    "Coloris foncé"
]
print("\nTests unitaires (phrases synthétiques) :")
for t in tests:
    cols = extract_colors_from_text(t)
    nu   = [to_nuance(c) for c in cols]
    print(f"- {t:25s} -> couleurs: {cols} | nuances: {nu}")


Tests unitaires (phrases synthétiques) :
- Chaussures bleu foncé     -> couleurs: ['bleu'] | nuances: ['bleu']
- Baskets vert clair        -> couleurs: ['vert'] | nuances: ['vert']
- Tennis rouge pastel       -> couleurs: ['rouge'] | nuances: ['rouge']
- Chaussures corail         -> couleurs: ['corail'] | nuances: ['orange']
- Baskets military          -> couleurs: ['military'] | nuances: ['vert']
- Sneakers navy             -> couleurs: ['navy'] | nuances: ['bleu']
- Juste clair               -> couleurs: ['clair'] | nuances: ['clair']
- Coloris foncé             -> couleurs: ['foncé'] | nuances: ['foncé']


In [23]:
print(df_colors.columns)

Index(['price_cur', 'price_init', 'discount', 'sale_duration', 'brand', 'name',
       'comment', 'shipping', 'image_url', 'url', 'gender_raw', 'brand_norm',
       'price_eur', 'price_cents', 'product_slug', 'genre_age', 'gender_db',
       'couleur', 'nuance'],
      dtype='object')


In [24]:
# nettoyage
def slugify_local(s: str) -> str:
    s = str(s) if s is not None else ""
    s = unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode().lower()
    s = re.sub(r"[^a-z0-9]+", "-", s).strip("-")
    return s

In [25]:
def build_model_name(name: str, brand_norm: str) -> str:
    t = name or ""
    if isinstance(brand_norm, str) and brand_norm:
        t = re.sub(rf"\b{re.escape(brand_norm)}\b", "", t, flags=re.IGNORECASE)
    # enlève quelques marques courantes (ajoute-en si besoin)
    t = re.sub(r"\b(nike|adidas|puma|new balance|nb|domyos|kiprun|asics|reebok|fila|salomon)\b",
               "", t, flags=re.IGNORECASE)
    # enlève les mots-couleurs si COLOR_PATTERN existe
    if 'COLOR_PATTERN' in globals():
        t = COLOR_PATTERN.sub("", t)
    t = re.sub(r"\s{2,}", " ", t).strip(" -–—_.")
    return t if t else (name or "")

In [26]:
# df_colors enrichissement
df_colors["model_name"] = df_colors.apply(lambda r: build_model_name(str(r["name"]), str(r["brand_norm"])), axis=1)
df_colors["model_slug"] = df_colors["model_name"].apply(slugify_local)

df_colors[["name","brand_norm","model_name","model_slug","couleur","nuance"]].head(8)


,name,brand_norm,model_name,model_slug,couleur,nuance
0,ADIDAS TERREX ANYLANDER IMPERMEABLE,Adidas,TERREX ANYLANDER IMPERMEABLE,terrex-anylander-impermeable,NA,NA
1,ADIDAS TERREX ANYLANDER IMPERMEABLE,Adidas,TERREX ANYLANDER IMPERMEABLE,terrex-anylander-impermeable,NA,NA
2,"Baksets homme, KLNJ BE D grise",Decathlon,"Baksets homme, KLNJ BE D",baksets-homme-klnj-be-d,grise,gris
3,BALLERINES DE MARCHE URBAINE FEMME PW 160 BR'E...,Decathlon,BALLERINES DE MARCHE URBAINE FEMME PW 160 BR'EASY,ballerines-de-marche-urbaine-femme-pw-160-br-easy,noir,noir
4,Basket de marche en toile femmeTBS Lescala beige,Tbs,Basket de marche en toile femmeTBS Lescala,basket-de-marche-en-toile-femmetbs-lescala,beige,beige
5,Basket de marche en toile femmeTBS Lescala marine,Tbs,Basket de marche en toile femmeTBS Lescala,basket-de-marche-en-toile-femmetbs-lescala,marine,bleu
6,"Basket de marche Femme, Flex Appeal 5.0 Fresh ...",Skechers,"Basket de marche Femme, Flex Appeal 5.0 Fresh ...",basket-de-marche-femme-flex-appeal-5-0-fresh-t...,NA,NA
7,"Basket de marche femme, Go walk Joy Violet Noir",Skechers,"Basket de marche femme, Go walk Joy",basket-de-marche-femme-go-walk-joy,violet,violet


In [27]:
data = df_colors.copy()
data[["name","brand_norm","model_name","model_slug","couleur","nuance"]].head(8)


,name,brand_norm,model_name,model_slug,couleur,nuance
0,ADIDAS TERREX ANYLANDER IMPERMEABLE,Adidas,TERREX ANYLANDER IMPERMEABLE,terrex-anylander-impermeable,NA,NA
1,ADIDAS TERREX ANYLANDER IMPERMEABLE,Adidas,TERREX ANYLANDER IMPERMEABLE,terrex-anylander-impermeable,NA,NA
2,"Baksets homme, KLNJ BE D grise",Decathlon,"Baksets homme, KLNJ BE D",baksets-homme-klnj-be-d,grise,gris
3,BALLERINES DE MARCHE URBAINE FEMME PW 160 BR'E...,Decathlon,BALLERINES DE MARCHE URBAINE FEMME PW 160 BR'EASY,ballerines-de-marche-urbaine-femme-pw-160-br-easy,noir,noir
4,Basket de marche en toile femmeTBS Lescala beige,Tbs,Basket de marche en toile femmeTBS Lescala,basket-de-marche-en-toile-femmetbs-lescala,beige,beige
5,Basket de marche en toile femmeTBS Lescala marine,Tbs,Basket de marche en toile femmeTBS Lescala,basket-de-marche-en-toile-femmetbs-lescala,marine,bleu
6,"Basket de marche Femme, Flex Appeal 5.0 Fresh ...",Skechers,"Basket de marche Femme, Flex Appeal 5.0 Fresh ...",basket-de-marche-femme-flex-appeal-5-0-fresh-t...,NA,NA
7,"Basket de marche femme, Go walk Joy Violet Noir",Skechers,"Basket de marche femme, Go walk Joy",basket-de-marche-femme-go-walk-joy,violet,violet


In [28]:
# Création des CSV
OUT_DIR = "/content/data_clean"
os.makedirs(OUT_DIR, exist_ok=True)

In [29]:
# clean_brands.csv
# Sécurité : on part du DF "data" (df_colors enrichi)
assert "data" in globals(), " (df_colors enrichi) défini plus haut."

brands = (data["brand_norm"]
          .fillna("")
          .map(lambda x: x.strip())
          .replace("", pd.NA)              # on ne garde pas les marques vides
          .dropna()
          .drop_duplicates()
          .rename("name")
          .to_frame())
brands["logo_url"] = ""

brands_path = f"{OUT_DIR}/clean_brands.csv"
brands.to_csv(brands_path, index=False)
print("✅ Écrit :", brands_path, "| lignes :", len(brands))

✅ Écrit : /content/data_clean/clean_brands.csv | lignes : 118


In [30]:
#clean_products.csv
# Fonctions utilitaires
def slugify_local(s: str) -> str:
    s = str(s) if s is not None else ""
    s = unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode().lower()
    s = re.sub(r"[^a-z0-9]+", "-", s).strip("-")
    return s

# sécurité: model_slug et model_name
if "model_slug" not in data.columns:
    data["model_slug"] = data["model_name"].apply(slugify_local)

stg_products = (
    data[["model_slug","model_name","brand_norm","price_eur","url","image_url","comment"]]
    .drop_duplicates(subset=["model_slug"])
    .rename(columns={
        "model_slug": "slug",
        "model_name": "name",
        "brand_norm": "brand",
        "price_eur":  "base_price",
        "url":        "scraped_from",
        "comment":    "description"   # on met le commentaire en description
    })
)

# forcer slug non vide
stg_products = stg_products[stg_products["slug"].notna() & (stg_products["slug"].astype(str).str.len() > 0)]

# ordre exact
stg_products = stg_products[["slug","name","brand","description","base_price","scraped_from","image_url"]]

products_path = f"{OUT_DIR}/clean_products.csv"
stg_products.to_csv(products_path, index=False)
print("✅ products →", products_path, "| lignes:", len(stg_products))


✅ products → /content/data_clean/clean_products.csv | lignes: 1991


In [31]:
#clean_product_images.csv
stg_images = (
    data[["model_slug","image_url"]]
    .dropna()
    .drop_duplicates()
    .rename(columns={"model_slug":"product_slug"})
)

# colonnes supplémentaires attendues par la staging
stg_images["alt_text"] = ""        # vide par défaut
stg_images["is_primary"] = True
stg_images["display_order"] = 0

# ordre exact
stg_images = stg_images[["product_slug","image_url","alt_text","is_primary","display_order"]]

images_path = f"{OUT_DIR}/clean_product_images.csv"
stg_images.to_csv(images_path, index=False)
print("✅ images →", images_path, "| lignes:", len(stg_images))

✅ images → /content/data_clean/clean_product_images.csv | lignes: 2211


In [39]:
# clean_product_variants.csv
# Revérification des cleaning
def slugify_local(s: str) -> str:
    s = str(s) if s is not None else ""
    s = unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode().lower()
    s = re.sub(r"[^a-z0-9]+", "-", s).strip("-")
    return s

def norm_txt(s: str) -> str:
    if not isinstance(s, str): return ""
    s = unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode().lower().strip()
    return s

def color_code(c: str, n: str) -> str:
    c = (c or "").lower(); n = (n or "").lower()
    mapping = {
        "blanc":"WHT","white":"WHT","noir":"BLK","black":"BLK",
        "bleu":"BLU","navy":"NVY","rose":"PNK","rouge":"RED","orange":"ORG",
        "jaune":"YLW","vert":"GRN","gris":"GRY","violet":"VLT","beige":"BEI",
        "marron":"BRN","metallique":"MET","multicolore":"MLT",
        "clair":"CLR","foncé":"DRK","pastel":"PST",
        "na":"NA", "unknown":"UNK","": "UNK", None:"UNK"
    }
    return mapping.get(n, mapping.get(c, "UNK"))

def make_sku_strong(brand, slug, color, color_family, size, gender):
    """
    SKU lisible + hash court. Jamais '-NAN-...'
    Format: BRAND4-SLUG14-COLORCODE-G{genderInitial}{size}-{HASH5}
    """
    b = (norm_txt(str(brand))[:4].upper() if pd.notna(brand) and str(brand) else "UNKB")
    s_short = (slugify_local(str(slug))[:14].upper() if pd.notna(slug) and str(slug) else "UNKS")
    cc = color_code(color, color_family)
    g = (str(gender)[:1] if gender else "X").upper()
    try:
        sz = int(float(size))
    except:
        sz = size if pd.notna(size) and str(size) else "UNK"
    sig_src = f"{brand}|{slug}|{color}|{color_family}|{size}|{gender}"
    hx = hashlib.sha1(sig_src.encode()).hexdigest()[:5].upper()
    return f"{b}-{s_short}-{cc}-{g}{sz}-{hx}"

In [40]:
# 1) Grille de tailles simple
size_grid = []
for sz in range(36, 42): size_grid.append(("woman", sz))
for sz in range(39, 49): size_grid.append(("man", sz))
for sz in range(28, 36): size_grid.append(("child", sz))
grid = pd.DataFrame(size_grid, columns=["gender","eu_size"])

# 2) Base variantes depuis 'data'
# On force les champs couleur/nuance à strings et 'NA' si vide
tmp = data.copy()
tmp["color"] = tmp.get("couleur", "NA").astype(str).fillna("NA").replace({"nan":"NA","None":"NA"})
tmp["color_family"] = tmp.get("nuance", "NA").astype(str).fillna("NA").replace({"nan":"NA","None":"NA"})
tmp["gender"] = tmp.get("gender_db", "").astype(str).str.lower()

# sécurité: model_slug et brand_norm
if "model_slug" not in tmp.columns:
    if "model_name" in tmp.columns:
        tmp["model_slug"] = tmp["model_name"].apply(slugify_local)
    else:
        raise RuntimeError("model_slug absent et model_name introuvable.")

tmp["brand"] = tmp.get("brand_norm", "").astype(str)

base_vars = (
    tmp[["model_slug","brand","gender","color","color_family","price_eur"]]
    .rename(columns={"price_eur":"price"})
)

# on enlève les lignes sans slug ou sans gender exploitable
base_vars = base_vars[
    base_vars["model_slug"].astype(str).str.len().gt(0) &
    base_vars["gender"].isin(["man","woman","child"])
]

# dédoublonnage niveau (modèle × couleur × genre)
base_vars = base_vars.drop_duplicates(subset=["model_slug","color","gender"])

# 3) Cross-join avec la grille de tailles par genre
variants = base_vars.merge(grid, on="gender", how="left")

# 4) Stock simulé stable et SKU robuste
variants["stock_quantity"] = 5 + (pd.util.hash_pandas_object(
    variants[["model_slug","gender","eu_size","color","color_family"]].astype(str)
).astype("int64") % 11)

variants["sku"] = variants.apply(
    lambda r: make_sku_strong(
        r["brand"], r["model_slug"], r["color"], r["color_family"], r["eu_size"], r["gender"]
    ),
    axis=1
)

# 5) Dédoublonner au niveau métier et vérifier l’unicité du SKU
before = len(variants)
variants = variants.drop_duplicates(subset=["model_slug","gender","eu_size","color","color_family"])
after = len(variants)
if after < before:
    print(f"ℹ️ Dédoublonnage variantes: {before} -> {after}")

dupe_count = variants["sku"].duplicated().sum()
if dupe_count > 0:
    print("⚠️ Collisions SKU restantes:", dupe_count)
    display(variants[variants["sku"].duplicated(keep=False)].head(20))
else:
    print("✅ Tous les SKU sont uniques.")

# 6) Colonnes et ordre EXACT pour la staging CSV
stg_variants = variants.rename(columns={"model_slug":"product_slug"})[
    ["product_slug","eu_size","gender","color","color_family","price","stock_quantity","sku"]
]

variants_path = f"{OUT_DIR}/clean_product_variants.csv"
stg_variants.to_csv(variants_path, index=False)
print("✅ variants →", variants_path, "| lignes:", len(stg_variants))
display(stg_variants.head(8))

✅ Tous les SKU sont uniques.
✅ variants → /content/data_clean/clean_product_variants.csv | lignes: 20704


,product_slug,eu_size,gender,color,color_family,price,stock_quantity,sku
0,terrex-anylander-impermeable,39,man,NA,NA,84.99,15,ADID-TERREX-ANYLAND-NA-M39-54BE8
1,terrex-anylander-impermeable,40,man,NA,NA,84.99,9,ADID-TERREX-ANYLAND-NA-M40-209BC
2,terrex-anylander-impermeable,41,man,NA,NA,84.99,7,ADID-TERREX-ANYLAND-NA-M41-CB23F
3,terrex-anylander-impermeable,42,man,NA,NA,84.99,15,ADID-TERREX-ANYLAND-NA-M42-B8181
4,terrex-anylander-impermeable,43,man,NA,NA,84.99,15,ADID-TERREX-ANYLAND-NA-M43-40633
5,terrex-anylander-impermeable,44,man,NA,NA,84.99,14,ADID-TERREX-ANYLAND-NA-M44-C9F11
6,terrex-anylander-impermeable,45,man,NA,NA,84.99,11,ADID-TERREX-ANYLAND-NA-M45-F04FD
7,terrex-anylander-impermeable,46,man,NA,NA,84.99,7,ADID-TERREX-ANYLAND-NA-M46-721BB


In [42]:
# 5) Contrôles
print("\nRécap fichiers :")
for p in [brands_path, products_path, images_path, variants_path]:
    print(" -", p)

# Checking
display(brands.head(3))
display(stg_products.head(3))
display(stg_images.head(3))
display(stg_variants.head(3))


Récap fichiers :
 - /content/data_clean/clean_brands.csv
 - /content/data_clean/clean_products.csv
 - /content/data_clean/clean_product_images.csv
 - /content/data_clean/clean_product_variants.csv


,name,logo_url
0,Adidas,
2,Decathlon,
4,Tbs,


,slug,name,brand,description,base_price,scraped_from,image_url
0,terrex-anylander-impermeable,TERREX ANYLANDER IMPERMEABLE,Adidas,(4),84.99,https://www.decathlon.fr/p/adidas-terrex-anyla...,https://contents.mediadecathlon.com/p2784230/k...
2,baksets-homme-klnj-be-d,"Baksets homme, KLNJ BE D",Decathlon,(934),18.99,https://www.decathlon.fr/p/baksets-homme-klnj-...,https://contents.mediadecathlon.com/p2633211/k...
3,ballerines-de-marche-urbaine-femme-pw-160-br-easy,BALLERINES DE MARCHE URBAINE FEMME PW 160 BR'EASY,Decathlon,(2671),26.99,https://www.decathlon.fr/p/ballerines-marche-s...,https://contents.mediadecathlon.com/p2394347/k...


,product_slug,image_url,alt_text,is_primary,display_order
0,terrex-anylander-impermeable,https://contents.mediadecathlon.com/p2784230/k...,,True,0
1,terrex-anylander-impermeable,https://contents.mediadecathlon.com/p2769590/k...,,True,0
2,baksets-homme-klnj-be-d,https://contents.mediadecathlon.com/p2633211/k...,,True,0


,product_slug,eu_size,gender,color,color_family,price,stock_quantity,sku
0,terrex-anylander-impermeable,39,man,NA,NA,84.99,15,ADID-TERREX-ANYLAND-NA-M39-54BE8
1,terrex-anylander-impermeable,40,man,NA,NA,84.99,9,ADID-TERREX-ANYLAND-NA-M40-209BC
2,terrex-anylander-impermeable,41,man,NA,NA,84.99,7,ADID-TERREX-ANYLAND-NA-M41-CB23F
